<a href="https://colab.research.google.com/github/xghan99/cs3244-team17/blob/main/CS3244_Random_Forest_%2B_BO%20(Baseline%20Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up



In [1]:
#Mounting Drive to access training data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Install scikit-optimize
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 6.6 MB/s 


In [22]:
#import libraries
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import pickle
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
import random
import math
from PIL import Image, ImageFile
from sklearn.preprocessing import StandardScaler
import pickle as pk  
from sklearn.decomposition import PCA
from skopt import BayesSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

## Downloading Data

In [ ]:
#Change dir to colab notebooks for downloading of dataset if needed
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
# Kaggle API to download dataset

#os.environ['KAGGLE_USERNAME'] = "" # username from the json file
#os.environ['KAGGLE_KEY'] = "" # key from the json file

In [ ]:
#!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
#!unzip new-plant-diseases-dataset

## Listing Classes

In [4]:
#change directory to train
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")

In [5]:
# Create list of classes
categories = os.listdir()
categories

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [6]:
# Sieveing out relevant classes
corn_diseases = categories[7:11]
potato_diseases = categories[20:23]
tomato_diseases = categories[28:]


In [7]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Corn Model

In [8]:
# Function for drawing out relevant training data 
# Code Source: https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01
def read_training_data(crop):
  flat_data_arr=[] #input array
  target_arr=[] #output array
  datadir='' #path which contains all the categories of images
  for i in crop:
    path=os.path.join(datadir,i)
    images = os.listdir(path)
    for img in images:
        try:
          img_array=imread(os.path.join(path,img))
          
        except AttributeError:
          continue
       
        img_resized=resize(img_array,(32,32,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(categories.index(i))
    
  
  flat_data=np.array(flat_data_arr)
  target=np.array(target_arr)
  df=pd.DataFrame(flat_data) #dataframe
  df['Target']=target
  x=df.iloc[:,:-1] #input data 
  y=df.iloc[:,-1] #output data
  return x,y


corn_train_x , corn_train_y = read_training_data(corn_diseases)


In [9]:
corn_train_x.shape

(7273, 3072)

In [10]:
# Scale and apply PCA on Data
# Code Source: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
def transform(x):
  scaler = StandardScaler()
  scaler.fit(x)
  x_new = scaler.transform(x)
  pca = PCA(0.95)
  pca.fit(x_new)
  x_new = pca.transform(x_new)

  return x_new, scaler, pca

corn_train_x_transformed, corn_scaler, corn_pca = transform(corn_train_x)

In [11]:
corn_train_x_transformed.shape

(7273, 145)

In [12]:
#Train Random Forest Classifier with BayesSearchCV
#Code Source: https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html
def train_model(train_x, train_y):
  opt = BayesSearchCV(
    RandomForestClassifier(),
    {
        'n_estimators': [80,100,140,200,280,380],
        'criterion': ['gini','entropy'],
        'max_features': ['auto', 'sqrt', 'log2'],  
        'max_depth': [10,15,25,40],
        'min_samples_split':[2,4,8],
        'min_samples_leaf':[1,2,5,10] 
    },
    n_iter=8,
    cv=3,
    scoring="accuracy",
    verbose=2,
    return_train_score=True
)

  opt.fit(train_x,train_y)

  return opt

corn_opt = train_model(corn_train_x_transformed, corn_train_y)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=entropy, max_depth=15, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=380; total time=  29.6s
[CV] END criterion=entropy, max_depth=15, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=380; total time=  29.6s
[CV] END criterion=entropy, max_depth=15, max_features=log2, min_samples_leaf=5, min_samples_split=2, n_estimators=380; total time=  29.8s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=entropy, max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=380; total time=  43.8s
[CV] END criterion=entropy, max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=380; total time=  43.5s
[CV] END criterion=entropy, max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=380; total time=  43.9s
Fitting 3 folds for each of 1 candidates, tot

In [13]:
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid")

In [14]:
def read_validation_data(crop, scaler, pca):
  flat_data_arr_val=[] #input array
  target_arr_val=[] #output array
  datadir='' 
  #path which contains all the categories of images
  for i in crop:
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        try:
          img_array=imread(os.path.join(path,img))
        except AttributeError:
          continue
        img_resized=resize(img_array,(32,32,3))
        flat_data_arr_val.append(img_resized.flatten())
        target_arr_val.append(categories.index(i))

  flat_data_val=np.array(flat_data_arr_val)
  target_val=np.array(target_arr_val)
  df_val=pd.DataFrame(flat_data_val) #dataframe
  df_val['Target']=target_val
  x_val=df_val.iloc[:,:-1] #input data 
  y_val=df_val.iloc[:,-1] #output data

  #Transform Data
  x_val = scaler.transform(x_val)
  x_val = pca.transform(x_val)

  return x_val,y_val

corn_val_x, corn_val_y = read_validation_data(corn_diseases, corn_scaler, corn_pca)

In [27]:
def evaluate(x_val,y_val,clf):
  y_predicted = clf.predict(x_val)
  score = accuracy_score(y_val,y_predicted)
  return score

evaluate(corn_val_x,corn_val_y, corn_opt)

0.8868234007654456

## Potato Model

In [16]:
#read_training_data
#transform
#train_model
#read_validation_data
#evaluate

In [17]:
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")
potato_train_x, potato_train_y = read_training_data(potato_diseases)
potato_train_x_transformed, potato_scaler, potato_pca = transform(potato_train_x)
potato_opt = train_model(potato_train_x_transformed,potato_train_y)
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid")
potato_val_x,potato_val_y = read_validation_data(potato_diseases, potato_scaler, potato_pca)



Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=gini, max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=8, n_estimators=380; total time=  11.8s
[CV] END criterion=gini, max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=8, n_estimators=380; total time=  13.2s
[CV] END criterion=gini, max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=8, n_estimators=380; total time=  11.5s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=gini, max_depth=15, max_features=log2, min_samples_leaf=10, min_samples_split=8, n_estimators=280; total time=   6.9s
[CV] END criterion=gini, max_depth=15, max_features=log2, min_samples_leaf=10, min_samples_split=8, n_estimators=280; total time=   6.9s
[CV] END criterion=gini, max_depth=15, max_features=log2, min_samples_leaf=10, min_samples_split=8, n_estimators=280; total time=   6.9s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]

0.8684397776171927

In [28]:
evaluate(potato_val_x,potato_val_y,potato_opt)

0.8674614305750351

## Tomato Model

In [19]:
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train")
tomato_train_x, tomato_train_y = read_training_data(tomato_diseases)
tomato_train_x_transformed, tomato_scaler, tomato_pca = transform(tomato_train_x)
tomato_opt = train_model(tomato_train_x_transformed,tomato_train_y)
os.chdir("/content/drive/My Drive/Colab Notebooks/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid")
tomato_val_x,tomato_val_y = read_validation_data(tomato_diseases, tomato_scaler, tomato_pca)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=140; total time= 1.4min
[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=140; total time= 1.4min
[CV] END criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=140; total time= 1.4min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=entropy, max_depth=25, max_features=sqrt, min_samples_leaf=10, min_samples_split=4, n_estimators=200; total time= 2.0min
[CV] END criterion=entropy, max_depth=25, max_features=sqrt, min_samples_leaf=10, min_samples_split=4, n_estimators=200; total time= 2.0min
[CV] END criterion=entropy, max_depth=25, max_features=sqrt, min_samples_leaf=10, min_samples_split=4, n_estimators=200; total time= 2.1min
Fitting 3 folds for each of 1 candidates, tot

0.71099295378366

In [31]:
evaluate(tomato_val_x,tomato_val_y,tomato_opt)

0.7131952017448201